In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import islice

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics.pairwise import cosine_similarity

path = '경로'

In [2]:
data = pd.read_excel(path + '/최종 모델링/최종 파일 (은서)/final_data.xlsx',index_col=0)
print(data.shape)
data.head()

(8520, 27)


,url,id,name,Artist,key_word,empty,sadness,enthusiasm,worry,love,...,sadness_a,enthusiasm_a,worry_a,love_a,fun_a,hate_a,happiness_a,boredom_a,relief_a,anger_a
0,https://open.spotify.com/track/0eFMbKCRw8KByXy...,0eFMbKCRw8KByXyWBw8WO7,Still With You,Jung Kook,face want see eye tell ecstatic memory even da...,0.016461,0.265974,0.067839,0.144616,0.116033,...,0.327877,0.102210,0.159917,0.434722,0.276396,0.002715,0.638312,0.095577,0.281033,0.006264
1,https://open.spotify.com/track/7eJMfftS33KTjuF...,7eJMfftS33KTjuF7lTsMCx,death bed (coffee for your head),Powfu,stay awake long go bed make cup coffee head ge...,0.122187,0.059084,0.039108,0.113228,0.024429,...,0.377954,0.120767,0.205617,0.359263,0.298656,0.002343,0.662892,0.098149,0.360254,0.005115
2,https://open.spotify.com/track/2CbGuO0LtVvbh3u...,2CbGuO0LtVvbh3umN3mDwM,Nan-Nan,Fujii Kaze,say didnt ask anything face w swear corner flo...,0.005816,0.248961,0.003242,0.483580,0.044740,...,0.340931,0.132052,0.188458,0.478974,0.359596,0.003094,0.745626,0.160948,0.443472,0.005347
3,https://open.spotify.com/track/1JRNrJKTQhfvxII...,1JRNrJKTQhfvxII8kkIcAQ,Mo-Eh-Wa,Fujii Kaze,night blown cold wind dont wander blow inner w...,0.077518,0.249271,0.011364,0.406069,0.011648,...,0.441021,0.070694,0.303113,0.262578,0.226800,0.001052,0.553588,0.048438,0.330071,0.002597
4,https://open.spotify.com/track/57u1FaZlztApkcT...,57u1FaZlztApkcT5x5XNuD,YASASHISA,Fujii Kaze,dream dream shade person kill kindness dream s...,0.001735,0.235752,0.001410,0.463885,0.142529,...,0.630868,0.055161,0.479071,0.189231,0.176755,0.001353,0.358413,0.049225,0.209036,0.005457


# API에서 도서 정보 들고오기

In [ ]:
import requests
import json

In [4]:
rest_api_key = "41d651c93152d5ec054dc828cacfa671"
url = "https://dapi.kakao.com/v3/search/book"
header = {"authorization": "KakaoAK "+rest_api_key}

In [5]:
#궁금한 도서의 isbn 입력
querynum = {"query": '어린왕자'}

In [6]:
#도서 정보 불러오기
response = requests.get(url, headers=header, params = querynum)
content = response.text
책정보 = json.loads(content)['documents'][0]

In [7]:
책정보

{'authors': ['생텍쥐페리'],
 'contents': '다른 별에서 온 어린 왕자의 순수한 시선으로 모순된 어른들의 세계를 비추는 전 세계가 사랑하는 아름다운 이야기 『어린 왕자』. 그동안 프랑스어 원문에 대한 섬세한 이해, 정확하고도 아름다운 문장력, 예리한 문학적 통찰을 고루 갖춘 번역으로 문학 번역에서 큰 입지를 굳혀 온 황현산. 그는 이 작품을 새롭게 번역하면서 생텍쥐페리의 진솔한 문체를 고스란히 살려 내기 위해 심혈을 기울였고, 원전의 가치를 충실히 살린 한국어 결정판을 마련하고자 했다.    이 작품',
 'datetime': '2015-10-20T00:00:00.000+09:00',
 'isbn': '8932917248 9788932917245',
 'price': 11800,
 'publisher': '열린책들',
 'sale_price': 10620,
 'status': '정상판매',
 'thumbnail': 'https://search1.kakaocdn.net/thumb/R120x174.q85/?fname=http%3A%2F%2Ft1.daumcdn.net%2Flbook%2Fimage%2F507587%3Ftimestamp%3D20231007143853',
 'title': '어린 왕자',
 'translators': ['황현산'],
 'url': 'https://search.daum.net/search?w=bookpage&bookId=507587&q=%EC%96%B4%EB%A6%B0+%EC%99%95%EC%9E%90'}

In [8]:
책정보['url']

'https://search.daum.net/search?w=bookpage&bookId=507587&q=%EC%96%B4%EB%A6%B0+%EC%99%95%EC%9E%90'

In [9]:
book = pd.DataFrame({'title': 책정보['title'],
              'isbn': 책정보['isbn'],
              'authors': 책정보['authors'],
              'publisher': 책정보['publisher']})
book

,title,isbn,authors,publisher
0,어린 왕자,8932917248 9788932917245,생텍쥐페리,열린책들


In [11]:
title = book['title']
title[0]

'어린 왕자'

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time

In [15]:
target_url = 책정보['url']
# 옵션 생성
options = webdriver.ChromeOptions()
# 창 숨기는 옵션 추가
options.add_argument("headless")

driver = webdriver.Chrome(options=options)
driver.get(target_url)
time.sleep(5)

try :
    botton = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[2]/div[3]/a')
    botton.click()
except :
    pass
책소개 = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[2]/p')

time.sleep(3)
try :
    botton = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[5]/div[3]/a')
    botton.click()
except :
    pass
책속으로 = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[5]/p')

time.sleep(3)
try :
    botton = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[6]/div[3]/a')
    botton.click()
except :
    pass
서평 = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[6]/p')

book['책소개'] = 책소개.text
book['책속으로'] = 책속으로.text
book['서평'] = 서평.text


In [16]:
img= driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[1]/div[1]/span/img')

In [17]:
img_src = img.get_attribute('src')
img_src

'https://search1.kakaocdn.net/thumb/C116x164.q85/?fname=http%3A%2F%2Ft1.daumcdn.net%2Flbook%2Fimage%2F507587%3Ftimestamp%3D20231007143853%3Fmoddttm=202401111202'

In [18]:
driver.close()

In [19]:
book

,title,isbn,authors,publisher,책소개,책속으로,서평
0,어린 왕자,8932917248 9788932917245,생텍쥐페리,열린책들,다른 별에서 온 어린 왕자의 순수한 시선으로 모순된 어른들의 세계를 비추는 전 세계...,"「내 생활은 단조로워. 나는 닭을 쫓고, 사람들은 나를 쫓고. 닭들은 모두 그게 그...","전 세계가 사랑하는 가장 아름다운 이야기, 생텍쥐페리의 『어린 왕자』가 문학 평론가..."


# 도서 설명 text 전처리

In [20]:
import googletrans
from googletrans import Translator

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer

import time

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\82109\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\82109\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\82109\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\82109\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
#영어 불용어 사전
stops = set(stopwords.words('english'))

def hapus_url(text):
    mention_pattern = r'@[\w]+'
    cleaned_text = re.sub(mention_pattern, '', text)
    return re.sub(r'http\S+','', cleaned_text)

#특수문자 제거
#영어 대소문자, 숫자, 공백문자(스페이스, 탭, 줄바꿈 등) 아닌 문자들 제거
def remove_special_characters(text, remove_digits=True):
    text=re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text


#불용어 제거
def delete_stops(text):
    text = text.lower().split()
    text = ' '.join([word for word in text if word not in stops])
    return text
   
    
#품사 tag 매칭용 함수
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    

#품사 태깅 + 표제어 추출
def tockenize(text):
    tokens=word_tokenize(text)
    pos_tokens=nltk.pos_tag(tokens)
    
    text_t=list()
    for _ in pos_tokens:
        text_t.append([_[0], get_wordnet_pos(_[1])])
    
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word[0], word[1]) for word in text_t])
    return text



def clean(text):
    text = remove_special_characters(text, remove_digits=True)
    text = delete_stops(text)
    text = tockenize(text)
    return text

In [23]:
translator = Translator()
for col in ['책소개', '책속으로', '서평']:
    name = col+'_trans'
    if book[col].values == '':
        book[name] = ''
        continue
    time.sleep(1)
    book[name] = clean(translator.translate(hapus_url(book.loc[0, col])).text)

book

,title,isbn,authors,publisher,책소개,책속으로,서평,책소개_trans,책속으로_trans,서평_trans
0,어린 왕자,8932917248 9788932917245,생텍쥐페리,열린책들,다른 별에서 온 어린 왕자의 순수한 시선으로 모순된 어른들의 세계를 비추는 전 세계...,"「내 생활은 단조로워. 나는 닭을 쫓고, 사람들은 나를 쫓고. 닭들은 모두 그게 그...","전 세계가 사랑하는 가장 아름다운 이야기, 생텍쥐페리의 『어린 왕자』가 문학 평론가...",beautiful story little prince love world adult...,life monotonousi chase chickens people chase m...,beautiful story world love little prince saint...


In [24]:
total_text = book.loc[0, '책소개_trans'] + book.loc[0, '책속으로_trans'] + book.loc[0, '서평_trans']
total_text

'beautiful story little prince love world adult contradict pure gaze young prince another starhwang hyun san solidified position literature translation translation french texture french text accurate beautiful sentiment sharp literary insightshe translate work try bring sincere style saint tech juperi prepare korean crystal version faithfully utilizes value nuclear power plant work remember novel full sentence power move mindthe translator hwang hyun san interpret source power sincere passionate style author saint tech zuperiyou enjoy work new complete translation melt deep reflection life beautiful metaphor give deep afterglowlife monotonousi chase chickens people chase methe chicken everyone thatso im boredbut tame life bright sunlightunlike footstep hear sound footprint sound differentlythe footprint hide groundyour footstep call cave like musicand see wheat fieldsi dont eat breadwheat use meso sadbut hair goldenso tame something really happenwheat golden remind youso love wind whea

# 도서 설명 text 감정 벡터 추출

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

df = pd.read_csv(path + '/최종 모델링/최종 파일 (은서)/tweet_data_agumentation.csv', index_col = 0)

tfidf_vect_emo = TfidfVectorizer()
tfidf_vect_emo.fit_transform(df["content"])

model = joblib.load(path + '/최종 모델링/최종 파일 (은서)/SVM.pkl')
total_text2 = tfidf_vect_emo.transform(pd.Series(total_text))
model.predict_proba(total_text2)
sentiment = pd.DataFrame(model.predict_proba(total_text2),index=['prob']).T
sentiment['감정'] = ['empty','sadness','enthusiasm','worry','love','fun','hate','happiness','boredom','relief','anger']
sentiment_sorted = sentiment.sort_values(by='prob',ascending=False)
sentiment2_sorted
# sentiment 이용!

,prob,감정
3,0.387811,worry
1,0.232665,sadness
2,0.098190,enthusiasm
7,0.097348,happiness
4,0.085197,love
5,0.044240,fun
9,0.037261,relief
6,0.011235,hate
0,0.004563,empty
8,0.000838,boredom


# audio feature 감정 벡터 - 도서 text 감정 벡터

In [26]:
audio_data = data.iloc[:,-11:]
audio_data

,empty_a,sadness_a,enthusiasm_a,worry_a,love_a,fun_a,hate_a,happiness_a,boredom_a,relief_a,anger_a
0,0.307403,0.327877,0.102210,0.159917,0.434722,0.276396,0.002715,0.638312,0.095577,0.281033,0.006264
1,0.354428,0.377954,0.120767,0.205617,0.359263,0.298656,0.002343,0.662892,0.098149,0.360254,0.005115
2,0.343030,0.340931,0.132052,0.188458,0.478974,0.359596,0.003094,0.745626,0.160948,0.443472,0.005347
3,0.357053,0.441021,0.070694,0.303113,0.262578,0.226800,0.001052,0.553588,0.048438,0.330071,0.002597
4,0.522309,0.630868,0.055161,0.479071,0.189231,0.176755,0.001353,0.358413,0.049225,0.209036,0.005457
...,...,...,...,...,...,...,...,...,...,...,...
8515,0.002945,0.001941,0.000195,0.000435,0.001216,0.000213,0.000248,0.000191,0.000539,0.998201,0.000589
8516,0.000520,0.000523,0.000260,0.000209,0.000297,0.000289,0.000164,0.000218,0.000410,0.999985,0.000158
8517,0.002854,0.001699,0.011960,0.000227,0.014427,0.011249,0.001502,0.008975,0.984361,0.001698,0.002366
8518,0.999991,0.999992,0.001036,0.000521,0.999909,0.000823,0.000659,0.000794,0.000225,0.000614,0.001103


In [28]:
sentiment

,prob,감정
0,0.004563,empty
1,0.232665,sadness
2,0.098190,enthusiasm
3,0.387811,worry
4,0.085197,love
5,0.044240,fun
6,0.011235,hate
7,0.097348,happiness
8,0.000838,boredom
9,0.037261,relief


In [29]:
sentiment_prob = sentiment['prob']
sentiment_prob.index = sentiment['감정']
sentiment_prob

감정
empty         0.004563
sadness       0.232665
enthusiasm    0.098190
worry         0.387811
love          0.085197
fun           0.044240
hate          0.011235
happiness     0.097348
boredom       0.000838
relief        0.037261
anger         0.000653
Name: prob, dtype: float64

In [30]:
audio_data.columns = ['empty', 'sadness', 'enthusiasm', 'worry', 'love', 'fun', 'hate',
       'happiness', 'boredom', 'relief', 'anger']

In [31]:
# 도서 감정 벡터를, af 감정 벡터 데이터프레임에 이어붙히기
audio_data_1 = pd.concat([sentiment_prob, audio_data.T],axis=1).T
audio_data_1

,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
prob,0.004563,0.232665,0.098190,0.387811,0.085197,0.044240,0.011235,0.097348,0.000838,0.037261,0.000653
0,0.307403,0.327877,0.102210,0.159917,0.434722,0.276396,0.002715,0.638312,0.095577,0.281033,0.006264
1,0.354428,0.377954,0.120767,0.205617,0.359263,0.298656,0.002343,0.662892,0.098149,0.360254,0.005115
2,0.343030,0.340931,0.132052,0.188458,0.478974,0.359596,0.003094,0.745626,0.160948,0.443472,0.005347
3,0.357053,0.441021,0.070694,0.303113,0.262578,0.226800,0.001052,0.553588,0.048438,0.330071,0.002597
...,...,...,...,...,...,...,...,...,...,...,...
8515,0.002945,0.001941,0.000195,0.000435,0.001216,0.000213,0.000248,0.000191,0.000539,0.998201,0.000589
8516,0.000520,0.000523,0.000260,0.000209,0.000297,0.000289,0.000164,0.000218,0.000410,0.999985,0.000158
8517,0.002854,0.001699,0.011960,0.000227,0.014427,0.011249,0.001502,0.008975,0.984361,0.001698,0.002366
8518,0.999991,0.999992,0.001036,0.000521,0.999909,0.000823,0.000659,0.000794,0.000225,0.000614,0.001103


In [32]:
col = ['book']+list(data['name'])

In [33]:
cosine_sim_audio = cosine_similarity(audio_data_1)
cosine_sim_audio_df = pd.DataFrame(cosine_sim_audio, index = col, columns=col)
cosine_sim_audio_df

,book,Still With You,death bed (coffee for your head),Nan-Nan,Mo-Eh-Wa,YASASHISA,Kiri Ga Naikara,Tsumi No Kaori,Cho Si Noccha Te,Tokuninai,...,My Heart Goes (La Di Da),Back To You,Tick Tack - Extended Mix,Turn Me Up - ViP Mix,Miserable Man,Murder in the City,September Song - Guitar Acoustic,Control,Heart's Content - Strings Mix,We Don't Have To Take Our Clothes Off
book,1.000000,0.561535,0.590789,0.541628,0.691217,0.785713,0.626805,0.536824,0.686589,0.196904,...,0.806499,0.077547,0.385614,0.077646,0.384877,0.078591,0.077565,0.010861,0.385031,0.079358
Still With You,0.561535,1.000000,0.992146,0.993036,0.958974,0.816293,0.915263,0.956427,0.911077,0.234024,...,0.171628,0.281828,0.619497,0.281836,0.618549,0.283573,0.281954,0.114190,0.618721,0.284791
death bed (coffee for your head),0.590789,0.992146,1.000000,0.992190,0.981064,0.853196,0.911203,0.930371,0.929398,0.238481,...,0.204913,0.340710,0.596371,0.340728,0.595376,0.342464,0.340822,0.110120,0.595569,0.343767
Nan-Nan,0.541628,0.993036,0.992190,1.000000,0.953863,0.793996,0.883576,0.924695,0.889078,0.222904,...,0.169406,0.372240,0.564096,0.372249,0.563115,0.373851,0.372371,0.153207,0.563314,0.375229
Mo-Eh-Wa,0.691217,0.958974,0.981064,0.953863,1.000000,0.929898,0.931265,0.908358,0.968080,0.252683,...,0.319004,0.338125,0.627395,0.338164,0.626406,0.340016,0.338204,0.064614,0.626610,0.341259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Murder in the City,0.078591,0.283573,0.342464,0.373851,0.340016,0.198484,0.199445,0.169891,0.240807,0.066934,...,0.001045,0.999995,0.004070,0.999995,0.003605,1.000000,0.999995,0.002303,0.003885,0.999981
September Song - Guitar Acoustic,0.077565,0.281954,0.340822,0.372371,0.338204,0.196214,0.197066,0.167863,0.238498,0.065946,...,0.000808,1.000000,0.001314,1.000000,0.000850,0.999995,1.000000,0.002151,0.001129,0.999971
Control,0.010861,0.114190,0.110120,0.153207,0.064614,0.057033,0.044308,0.038633,0.078931,0.012020,...,0.000732,0.001946,0.011240,0.002050,0.011165,0.002303,0.002151,1.000000,0.011278,0.008262
Heart's Content - Strings Mix,0.385031,0.618721,0.595569,0.563314,0.626610,0.724787,0.861851,0.798720,0.785588,0.244841,...,0.006679,0.001054,0.999999,0.001071,1.000000,0.003885,0.001129,0.011278,1.000000,0.005067


In [34]:
audio_sim = cosine_sim_audio_df['book']
audio_sim

book                                     1.000000
Still With You                           0.561535
death bed (coffee for your head)         0.590789
Nan-Nan                                  0.541628
Mo-Eh-Wa                                 0.691217
                                           ...   
Murder in the City                       0.078591
September Song - Guitar Acoustic         0.077565
Control                                  0.010861
Heart's Content - Strings Mix            0.385031
We Don't Have To Take Our Clothes Off    0.079358
Name: book, Length: 8521, dtype: float64

# 가사 감정 벡터 - 도서 text 감정 벡터

In [35]:
lyrics_data = data.iloc[:,5:-11]
lyrics_data

,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,0.016461,0.265974,0.067839,0.144616,0.116033,0.206217,0.015079,0.134595,0.002582,0.028249,0.002356
1,0.122187,0.059084,0.039108,0.113228,0.024429,0.023013,0.015683,0.212476,0.004940,0.384384,0.001468
2,0.005816,0.248961,0.003242,0.483580,0.044740,0.008447,0.022421,0.152159,0.000405,0.029745,0.000483
3,0.077518,0.249271,0.011364,0.406069,0.011648,0.038887,0.005062,0.123563,0.002235,0.072786,0.001598
4,0.001735,0.235752,0.001410,0.463885,0.142529,0.009176,0.010140,0.120540,0.000858,0.013238,0.000737
...,...,...,...,...,...,...,...,...,...,...,...
8515,0.011245,0.162807,0.006966,0.456847,0.082303,0.040452,0.002504,0.231635,0.000722,0.004143,0.000376
8516,0.021068,0.348507,0.035223,0.432478,0.046740,0.014127,0.007841,0.074129,0.001142,0.017115,0.001629
8517,0.002924,0.170607,0.004646,0.211214,0.003779,0.007508,0.575163,0.020068,0.000557,0.002283,0.001252
8518,0.004582,0.297325,0.004344,0.583962,0.050943,0.004454,0.001543,0.051992,0.000209,0.000479,0.000167


In [36]:
lyrics_data.columns

Index(['empty', 'sadness', 'enthusiasm', 'worry', 'love', 'fun', 'hate',
       'happiness', 'boredom', 'relief', 'anger'],
      dtype='object')

In [37]:
# 도서 감정 벡터를, 가사 감정 벡터 데이터프레임에 이어붙히기
lyrics_data_1 = pd.concat([sentiment_prob, lyrics_data.T],axis=1).T
lyrics_data_1 

,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
prob,0.004563,0.232665,0.098190,0.387811,0.085197,0.044240,0.011235,0.097348,0.000838,0.037261,0.000653
0,0.016461,0.265974,0.067839,0.144616,0.116033,0.206217,0.015079,0.134595,0.002582,0.028249,0.002356
1,0.122187,0.059084,0.039108,0.113228,0.024429,0.023013,0.015683,0.212476,0.004940,0.384384,0.001468
2,0.005816,0.248961,0.003242,0.483580,0.044740,0.008447,0.022421,0.152159,0.000405,0.029745,0.000483
3,0.077518,0.249271,0.011364,0.406069,0.011648,0.038887,0.005062,0.123563,0.002235,0.072786,0.001598
...,...,...,...,...,...,...,...,...,...,...,...
8515,0.011245,0.162807,0.006966,0.456847,0.082303,0.040452,0.002504,0.231635,0.000722,0.004143,0.000376
8516,0.021068,0.348507,0.035223,0.432478,0.046740,0.014127,0.007841,0.074129,0.001142,0.017115,0.001629
8517,0.002924,0.170607,0.004646,0.211214,0.003779,0.007508,0.575163,0.020068,0.000557,0.002283,0.001252
8518,0.004582,0.297325,0.004344,0.583962,0.050943,0.004454,0.001543,0.051992,0.000209,0.000479,0.000167


In [38]:
col = ['book']+list(data['name'])

In [39]:
cosine_sim_lyrics = cosine_similarity(lyrics_data_1)
cosine_sim_lyrics_df = pd.DataFrame(cosine_sim_lyrics, index =col, columns=col)
cosine_sim_lyrics_df

,book,Still With You,death bed (coffee for your head),Nan-Nan,Mo-Eh-Wa,YASASHISA,Kiri Ga Naikara,Tsumi No Kaori,Cho Si Noccha Te,Tokuninai,...,My Heart Goes (La Di Da),Back To You,Tick Tack - Extended Mix,Turn Me Up - ViP Mix,Miserable Man,Murder in the City,September Song - Guitar Acoustic,Control,Heart's Content - Strings Mix,We Don't Have To Take Our Clothes Off
book,1.000000,0.787840,0.434940,0.968089,0.958215,0.970050,0.977434,0.934927,0.973198,0.975436,...,0.892648,0.594580,0.469537,0.723996,0.947830,0.936591,0.969668,0.425899,0.957680,0.946887
Still With You,0.787840,1.000000,0.425674,0.701512,0.741244,0.720355,0.736091,0.579968,0.741391,0.806184,...,0.770901,0.681153,0.452202,0.860433,0.645661,0.704713,0.756607,0.340225,0.650681,0.805498
death bed (coffee for your head),0.434940,0.425674,1.000000,0.427589,0.523732,0.384234,0.418146,0.322419,0.463423,0.454470,...,0.289112,0.265163,0.148591,0.486258,0.286708,0.449359,0.362375,0.161462,0.308787,0.410427
Nan-Nan,0.968089,0.701512,0.427589,1.000000,0.978559,0.981930,0.994108,0.970731,0.981328,0.981550,...,0.887594,0.512962,0.388462,0.670581,0.964985,0.971782,0.968385,0.445183,0.979399,0.934484
Mo-Eh-Wa,0.958215,0.741244,0.523732,0.978559,1.000000,0.948702,0.975881,0.945951,0.952311,0.964883,...,0.898202,0.495309,0.327175,0.747536,0.949936,0.937856,0.968041,0.418077,0.956923,0.908138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Murder in the City,0.936591,0.704713,0.449359,0.971782,0.937856,0.962677,0.963903,0.923977,0.988209,0.966096,...,0.785445,0.504854,0.442816,0.689337,0.888574,1.000000,0.898331,0.377529,0.922304,0.913763
September Song - Guitar Acoustic,0.969668,0.756607,0.362375,0.968385,0.968041,0.957601,0.976385,0.939499,0.935785,0.965289,...,0.966941,0.572421,0.380675,0.710821,0.978626,0.898331,1.000000,0.438148,0.974910,0.935146
Control,0.425899,0.340225,0.161462,0.445183,0.418077,0.417850,0.447162,0.431568,0.407786,0.440859,...,0.442076,0.395134,0.137649,0.304154,0.439843,0.377529,0.438148,1.000000,0.420019,0.494162
Heart's Content - Strings Mix,0.957680,0.650681,0.308787,0.979399,0.956923,0.972185,0.980372,0.989085,0.940497,0.947384,...,0.900202,0.496447,0.394319,0.623342,0.995915,0.922304,0.974910,0.420019,1.000000,0.909440


In [40]:
lyrics_sim = cosine_sim_lyrics_df['book']
lyrics_sim

book                                     1.000000
Still With You                           0.787840
death bed (coffee for your head)         0.434940
Nan-Nan                                  0.968089
Mo-Eh-Wa                                 0.958215
                                           ...   
Murder in the City                       0.936591
September Song - Guitar Acoustic         0.969668
Control                                  0.425899
Heart's Content - Strings Mix            0.957680
We Don't Have To Take Our Clothes Off    0.946887
Name: book, Length: 8521, dtype: float64

# 가사 내용 - 도서 text 키워드 유사도

In [41]:
keyword_data = data['key_word']
keyword_data

0       face want see eye tell ecstatic memory even da...
1       stay awake long go bed make cup coffee head ge...
2       say didnt ask anything face w swear corner flo...
3       night blown cold wind dont wander blow inner w...
4       dream dream shade person kill kindness dream s...
                              ...                        
8515    get murder city revenge name one person dead p...
8516    innocent darling use talk til morning mixtape ...
8517    wan na lose control even though old know shake...
8518    love live without u one thing know sure love f...
8519    take clothes good time oh oh could dance party...
Name: key_word, Length: 8520, dtype: object

In [42]:
total_text

'beautiful story little prince love world adult contradict pure gaze young prince another starhwang hyun san solidified position literature translation translation french texture french text accurate beautiful sentiment sharp literary insightshe translate work try bring sincere style saint tech juperi prepare korean crystal version faithfully utilizes value nuclear power plant work remember novel full sentence power move mindthe translator hwang hyun san interpret source power sincere passionate style author saint tech zuperiyou enjoy work new complete translation melt deep reflection life beautiful metaphor give deep afterglowlife monotonousi chase chickens people chase methe chicken everyone thatso im boredbut tame life bright sunlightunlike footstep hear sound footprint sound differentlythe footprint hide groundyour footstep call cave like musicand see wheat fieldsi dont eat breadwheat use meso sadbut hair goldenso tame something really happenwheat golden remind youso love wind whea

In [43]:
book_song_cont1 = pd.DataFrame({"text": total_text}, index = range(1))
book_song_cont1

,text
0,beautiful story little prince love world adult...


In [44]:
book_song_cont2 = pd.DataFrame({"text": keyword_data})
book_song_cont2

,text
0,face want see eye tell ecstatic memory even da...
1,stay awake long go bed make cup coffee head ge...
2,say didnt ask anything face w swear corner flo...
3,night blown cold wind dont wander blow inner w...
4,dream dream shade person kill kindness dream s...
...,...
8515,get murder city revenge name one person dead p...
8516,innocent darling use talk til morning mixtape ...
8517,wan na lose control even though old know shake...
8518,love live without u one thing know sure love f...


In [125]:
keyword_data_1 = pd.concat([book_song_cont1, book_song_cont2], axis=0).reset_index(drop=True)
keyword_data_1

,text
0,beautiful story little prince love world adult...
1,face want see eye tell ecstatic memory even da...
2,stay awake long go bed make cup coffee head ge...
3,say didnt ask anything face w swear corner flo...
4,night blown cold wind dont wander blow inner w...
...,...
8516,get murder city revenge name one person dead p...
8517,innocent darling use talk til morning mixtape ...
8518,wan na lose control even though old know shake...
8519,love live without u one thing know sure love f...


In [126]:
tfidf_vect_cont = TfidfVectorizer()
tfidf_matrix_cont = tfidf_vect_cont.fit_transform(keyword_data_1['text'])
tfidf_array_cont = tfidf_matrix_cont.toarray()
tfidf_df_cont = pd.DataFrame(tfidf_array_cont, columns=tfidf_vect_cont.get_feature_names_out())
tfidf_df_cont

,00,000,00s,01,03,04,050,06,07,085,...,zun,zunk,zuperi,zuperiit,zuperis,zuperiyou,zurna,zyaada,zyla,zzoil
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.05689,0.018963,0.018963,0.018963,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
8517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
8518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
8519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0


In [127]:
col = ['book']+list(data['name'])

In [128]:
cosine_sim_keyword = cosine_similarity(tfidf_array_cont)
cosine_sim_keyword_df = pd.DataFrame(cosine_sim_keyword, index = col, columns=col)
cosine_sim_keyword_df

In [ ]:
keyword_sim = cosine_sim_keyword_df['book']
keyword_sim

book                                     1.000000
Still With You                           0.052323
death bed (coffee for your head)         0.039123
Nan-Nan                                  0.041644
Mo-Eh-Wa                                 0.015949
                                           ...   
Murder in the City                       0.044782
September Song - Guitar Acoustic         0.000000
Control                                  0.033150
Heart's Content - Strings Mix            0.030498
We Don't Have To Take Our Clothes Off    0.032305
Name: book, Length: 8521, dtype: float64

# 전체 유사도 계산

In [ ]:
total_sim  = 0.8*audio_sim + 0.1*lyrics_sim + 0.1*keyword_sim

recommend_song = total_sim.sort_values(ascending=False)[1:6].index
total_sim_df = pd.DataFrame(total_sim[1:])
total_sim_df = total_sim_df.reset_index()
total_sim_df.columns = ['name','book']

top_five = total_sim_df.sort_values(by='book',ascending=False)[:5]
index = total_sim_df.sort_values(by='book',ascending=False)[:5].index.sort_values()

In [ ]:
artist = data.iloc[index][['url','name','Artist']]
top_five_df = pd.merge(artist,top_five,on='name').sort_values(by='book',ascending=False).drop_duplicates()
top_five_df 

,url,name,Artist,book
0,https://open.spotify.com/track/1JRNrJKTQhfvxII...,Mo-Eh-Wa,Fujii Kaze,0.748844
3,https://open.spotify.com/track/2sOOXgEGB7nhKO8...,Dominic’s Interlude,Halsey,0.748208
1,https://open.spotify.com/track/57u1FaZlztApkcT...,YASASHISA,Fujii Kaze,0.747754
2,https://open.spotify.com/track/6BORJ0ZazxMBpM3...,Forever ... (is a long time),Halsey,0.743526
4,https://open.spotify.com/track/6FZDfxM3a3UCqtz...,Without Me,Halsey,0.742754


In [ ]:
total_sim_2  = 0*audio_sim + 0.5*lyrics_sim + 0.5*keyword_sim

recommend_song = total_sim_2.sort_values(ascending=False)[1:6].index
total_sim_df_2 = pd.DataFrame(total_sim_2[1:])
total_sim_df_2 = total_sim_df_2.reset_index()
total_sim_df_2.columns = ['name','book']

top_five_2 = total_sim_df_2.sort_values(by='book',ascending=False)[:5]
index = total_sim_df_2.sort_values(by='book',ascending=False)[:5].index.sort_values()

In [ ]:
artist = data.iloc[index][['url','name','Artist']]
top_five_df_2 = pd.merge(artist,top_five_2,on='name').sort_values(by='book',ascending=False).drop_duplicates()
top_five_df_2

,url,name,Artist,book
1,https://open.spotify.com/track/4ASOMWcJUrwEpjJ...,California Dreamin' (feat. High Jinx),Chris Lorenzo,0.530188
3,https://open.spotify.com/track/3kcKlOkQQEPVwxw...,Kernkraft 400 (A Better Day),Topic,0.525266
4,https://open.spotify.com/track/6PBW6fyJe4YzKhC...,Last Christmas,Various Artists,0.522569
0,https://open.spotify.com/track/04J9aoBIvtKSeDC...,Back It Up,Lil Tecca,0.521334
2,https://open.spotify.com/track/5WYTSyhKonzQZ3h...,もしも命が描けたら,YOASOBI,0.518032


In [ ]:
url_list = top_five_df['url']
url_list

0    https://open.spotify.com/track/1JRNrJKTQhfvxII...
3    https://open.spotify.com/track/2sOOXgEGB7nhKO8...
1    https://open.spotify.com/track/57u1FaZlztApkcT...
2    https://open.spotify.com/track/6BORJ0ZazxMBpM3...
4    https://open.spotify.com/track/6FZDfxM3a3UCqtz...
Name: url, dtype: object

In [ ]:
top_five_df.iloc[1]

url       https://open.spotify.com/track/2sOOXgEGB7nhKO8...
name                                    Dominic’s Interlude
Artist                                               Halsey
book                                               0.748208
Name: 3, dtype: object

In [ ]:
print('TOP 1\n')
print('제목 : {0} \n가수 : {1} \nurl : {2} \n유사도 : {3:.4f}'.format(top_five_df.iloc[0]['name'],top_five_df.iloc[0]['Artist'],top_five_df.iloc[0]['url'],top_five_df.iloc[0]['book']))

TOP 1

제목 : Mo-Eh-Wa 
가수 : Fujii Kaze 
url : https://open.spotify.com/track/1JRNrJKTQhfvxII8kkIcAQ 
유사도 : 0.7488
